In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re
from pathlib import Path



In [8]:
%pip install -U langchain langchain-community


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 465.6 kB/s eta 0:00:05
   -------- ------------------------------- 0.5/2.5 MB 465.6 kB/s eta 0:00:05
   ------------ --------------------------- 0.8/2.5 MB 479.2 kB/s eta 0:00:04
   ------------ --------------------------- 0.8/2.5 MB 479.2 kB/s eta 0:00:04
   ------------ --------------------------- 0.8/2.5 MB 479


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:

df = pd.read_csv(
    r"C:\Users\jkk\OneDrive\Desktop\rag-complaint-chatbot\data\processed\filtered_complaints.csv"
)

In [5]:
sampled_df = (
    df.groupby("product", group_keys=False)
      .apply(lambda x: x.sample(n=3000, random_state=42))
)

sampled_df["product"].value_counts()


product
Credit card    3000
Name: count, dtype: int64

In [16]:
def chunk_text(text, chunk_size=500, overlap=50):
    """
    Splits text into overlapping character chunks.
    """
    chunks = []
    start = 0
    text_length = len(text)

    while start < text_length:
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start = end - overlap

    return chunks


In [17]:
documents = []

for _, row in sampled_df.iterrows():
    chunks = chunk_text(row["clean_text"])
    for i, chunk in enumerate(chunks):
        documents.append({
            "text": chunk,
            "complaint_id": row["complaint_id"],
            "product": row["product"],
            "issue": row["issue"],
            "sub_issue": row["sub-issue"],
            "chunk_index": i
        })

len(documents)


8824

In [19]:
%pip install sentence-transformers

from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

texts = [doc["text"] for doc in documents]
embeddings = model.encode(texts, show_progress_bar=True)


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/110.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/110.9 MB ? eta -:--:--
   ---------------------------------------- 0.5/110.9 MB 3.4 MB/s eta 0:00:33
   ---------------------------------------- 0.5/110.9 MB 3.4 MB/s eta 0:00:33
   ---------------------------------------- 0.8/110.9 MB 1.2 MB/s eta 0:01:29
   ---------------------------------------- 0.8/110.9 MB 1.2 MB/s eta 0:01:29
   ---------------------------------------- 1.0/110.9 MB 811.6 kB/s eta 0:02:16
   ---------------------------------------- 1.0/110.9 MB 811.6 kB/s eta 0:02:16
   ---------------------------------------- 1.0/110.9 MB 811.6 kB/s eta 0:02:16
   ---------------------------------------- 1.0/110.9 MB 811.6 kB/s eta 0:02:16
   ---------------------------------------- 1.3/110.9 MB 568.6 kB/s eta 0:03:13
   ---------------------------------------- 1.3/110.9 MB 568.6 kB/s 


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/276 [00:00<?, ?it/s]

In [21]:
%pip install faiss-cpu

import faiss
import numpy as np

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/18.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.9 MB ? eta -:--:--
    --------------------------------------- 0.3/18.9 MB ? eta -:--:--
   - -------------------------------------- 0.5/18.9 MB 699.0 kB/s eta 0:00:27
   - -------------------------------------- 0.5/18.9 MB 699.0 kB/s eta 0:00:27
   - -------------------------------------- 0.8/18.9 MB 780.2 kB/s eta 0:00:24
   -- ------------------------------------- 1.0/18.9 MB 838.4 kB/s eta 0:00:22
   -- ------------------------------------- 1.3/18.9 MB 882.6 kB/s eta 0:00:20
   --- ------------------------------------ 1.6/18.9 MB 931.6 kB/s eta 0:00:19
   --- ------------------------------------ 1.8/18.9 MB 958.5 kB/s eta 0:00:18
   --- ------------------------------------ 1.8/18.9 MB 958.5 kB/s eta 0:00:18
   ---- --

In [22]:
import pickle
import os

vector_dir = r"C:\Users\jkk\OneDrive\Desktop\rag-complaint-chatbot\vector_store"
os.makedirs(vector_dir, exist_ok=True)

faiss.write_index(index, os.path.join(vector_dir, "complaints_sample.index"))

with open(os.path.join(vector_dir, "metadata_sample.pkl"), "wb") as f:
    pickle.dump(documents, f)
